# Analysing Audioscrobbler Data

In this notebook we want to create two very simple statistics on artists from data provided by Audioscrobbler. We are working with three related data sets:

1. A list of users containing their number of plays per artist
2. A list which maps a generic artist-id to its real (band) name
3. A list which maps bad artist ids to good ones (fixing typing errors)

Then we will ask two simple questions:

1. Which artists have the most listeners (in terms of number of unique users)
2. Which artists are played most often (in terms of total play counts)

# 1 Load Data

First of all we have to load the data from S3.

## 1.1 Read User-Artist Data

First we read in the most important data set, containing the information how often a user played songs of a specific artist. This information is stored in the file at `s3://dimajix-training/data/audioscrobbler/user_artist_data/`. The file has the following characteristics:

* Format: CSV (kind of)
* Separator: Space (” “)
* Header: No
* Fields: user_id (INT), artist_id(INT), play_count(INT)

So we need to read in this file and store it in a local variable user_artist_data. Since we do not have any header contained in the file itself, we need to specify the schema explicitly.

In [1]:
from pyspark.sql.types import *

schema = StructType([
        StructField("user_id", IntegerType()),
        StructField("artist_id", IntegerType()),
        StructField("play_count", IntegerType())
    ])
    
user_artist_data = spark.read \
    .option("sep", " ") \
    .option("header", False) \
    .schema(schema) \
    .csv("s3://dimajix-training/data/audioscrobbler/user_artist_data/")

### Peek inside
Let us have a look at the first 5 records

In [2]:
user_artist_data.limit(5).toPandas()

,user_id,artist_id,play_count
0,1000002,1,55
1,1000002,1000006,33
2,1000002,1000007,8
3,1000002,1000009,144
4,1000002,1000010,314


## 1.2 Read in Artist aliases

Now we read in a file containing mapping of bad artist IDs to good IDs. This fixes typos in the artists names and thereby enables us to merge information with different artist IDs belonging to the same band. This information is stored in the file at `s3://dimajix-training/data/audioscrobbler/artist_alias/`. The file has the following characteristics:

* Format: CSV (kind of)
* Separator: Tab (“\t”)
* Header: No
* Fields: bad_id (INT), good_id(INT)

So we need to read in this file and store it in a local variable `artist_alias`. Since we do not have any header contained in the file itself, we need to specify the schema explicitly.

In [3]:
schema = StructType([
        StructField("bad_id", IntegerType()),
        StructField("good_id", IntegerType())
    ])
    
artist_alias = spark.read \
    .option("sep","\t") \
    .option("header", False) \
    .schema(schema) \
    .csv("s3://dimajix-training/data/audioscrobbler/artist_alias/")

### Peek inside

In [4]:
artist_alias.limit(5).toPandas()

,bad_id,good_id
0,1092764,1000311
1,1095122,1000557
2,6708070,1007267
3,10088054,1042317
4,1195917,1042317


## 1.3 Read in Artist names

The third file contains the artists name for his/her id. We also use this file in order to be able to display results with the artists names instead of their IDs. This information is stored in the file at `s3://dimajix-training/data/audioscrobbler/artist_data/`. The file has the following characteristics:

* Format: CSV (kind of)
* Separator: Tab (“\t”)
* Header: No
* Fields: artist_id (INT), artist_name(STRING)

So we need to read in this file and store it in a local variable `artist_data`. Since we do not have any header contained in the file itself, we need to specify the schema explicitly.

In [6]:
schema = StructType([
        StructField("artist_id", IntegerType()),
        StructField("artist_name", StringType())
    ])

artist_data = spark.read \
    .option("sep","\t") \
    .option("header",False) \
    .schema(schema) \
    .csv("s3://dimajix-training/data/audioscrobbler/artist_data/")

### Peek inside

In [7]:
artist_data.limit(5).toPandas()

,artist_id,artist_name
0,1134999,06Crazy Life
1,6821360,Pang Nakarin
2,10113088,"Terfel, Bartoli- Mozart: Don"
3,10151459,The Flaming Sidebur
4,6826647,Bodenstandig 3000


# 2 Clean Data

Before continuing with the analysis, we first create a cleaned version of the `artist_user_data` DataFrame with the `artist_alias` mapping applied. This means that we need to lookup each artist-id in the original data set in the alias data set and see if we find have a matching `bad_id` entry with a replacement `good_id`. The result should be stored in a variable `cleaned_user_artist_data`. This DataFrame should contain the columns Finally select only the columns `user_id`, `artist_id` (the corrected one) and `play_count`.

Hints:

1. Join the user artist data DataFrame with the artist alias DataFrame containing fixes for some artists. Which join type is appropriate?    
2. Replace the artist-id in the user artists data with the `good_id` from the artist alias DataFrame - if a match is found on `bad_id`    
3. Finally select only the columns `user_id`, `artist_id` (the corrected one) and `play_count`

In [8]:
from pyspark.sql.functions import *

cleaned_user_artist_data = user_artist_data.join(artist_alias, user_artist_data["artist_id"] == artist_alias["bad_id"], "left") \
    .select(
        user_artist_data["user_id"], 
        coalesce(artist_alias["good_id"], user_artist_data["artist_id"]).alias("artist_id"), 
        user_artist_data["play_count"]
    )
    
cleaned_user_artist_data.limit(5).toPandas()    

,user_id,artist_id,play_count
0,1000002,1,55
1,1000002,1000006,33
2,1000002,1000007,8
3,1000002,1000009,144
4,1000002,1000010,314


# 3 Question 1: Artist with most unique listeners

Who are the artist with the most unique listeners? For this question, it is irrelevant how often an individul user has played songs of a specific artist. It is only important how many different users have played each artists work. Of course we do not want to see the artist-id, but their real names as provided in the DataFrame `artist_data`!

Hints:
1. Group cleaned data by `artist_id`
2. Perform aggregation by counting unique user ids
3. Join `artist_data`
4. Lookup the artists name
5. Sort result by descending user counts

In [9]:
result = cleaned_user_artist_data \
    .groupBy("artist_id") \
    .agg(countDistinct(col("user_id")).alias("user_count")) \
    .join(artist_data, cleaned_user_artist_data["artist_id"] == artist_data["artist_id"]) \
    .select(artist_data["artist_name"], col("user_count")) \
    .orderBy(col("user_count").desc())

result.limit(10).toPandas()

,artist_name,user_count
0,Radiohead,45630
1,Green Day,43883
2,The Beatles,40919
3,Coldplay,38497
4,Nirvana,36823
5,Red Hot Chili Peppers,36036
6,U2,34853
7,Metallica,32337
8,Weezer,31951
9,[unknown],31324


# 4 Question 2: Artist with most Plays

Now we also take into account how often each user played the work of individual artists. That is, we also include the `play_coun` column into our calculations. So which artists have the most plays in total? Of course we do not want to see the artist-id, but their real names as provided in the DataFrame `artist_data`!

Hints:
1. Group cleaned data by `artist_id`
2. Perform aggregation by summing up play count
3. Join `artist_data`
4. Lookup the artists name
5. Sort result by descending play counts


In [11]:
result = cleaned_user_artist_data \
    .groupBy("artist_id") \
    .agg(sum(col("play_count")).alias("play_count")) \
    .join(artist_data, cleaned_user_artist_data["artist_id"] == artist_data["artist_id"]) \
    .select(artist_data["artist_name"], col("play_count")) \
    .orderBy(col("play_count").desc())

result.limit(5).toPandas()

,artist_name,play_count
0,Radiohead,2502596
1,The Beatles,2259825
2,Green Day,1931143
3,Metallica,1543430
4,System of a Down,1426254
